# Jupyter Notebook with Java

Run the following code block using a Python kernel. It installs jbang and java kernel.

In [ ]:
!pip3 install jbang
import jbang
jbang.exec("trust add https://github.com/jupyter-java")
print(jbang.exec("install-kernel@jupyter-java --java 21 --enable-preview").stdout)

# Switch to Java kernel

When the above have completed you should be able to switch to using a Java kernel (some call it runtime) in your Jupyter notebook environment.

You might need to install the Jupyter python package in some environments - particular VSCode based IDE's might not have it directly available.



In [ ]:
!pip install jupyter

When installed try switch kernels. Might have to reload your IDE to have it detect the new available kernels.

# Test it works

Run the following code using a Java kernel

In [1]:
// ADD DEPENDENCIES
%classpath /workspaces/cyoda-client-example/accounting-demo-app/ipynb

import com.example.accounting_demo.service.RmiEntityService;
import com.example.accounting_demo.dto.SerializableHttpResponse;

In [29]:
// CREATE APP CONFIG AND LAUNCHER
import java.io.File;
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.Paths;
import java.util.List;

public class RunSpringBootApp {

    private static Process process;

    public static void startApp() {
        String jarPath = "application.jar";
        File logFile = new File("application.log"); // Log file
        ProcessBuilder processBuilder = new ProcessBuilder("java", "-jar", jarPath);
        
        // Redirect output and errors to a file
        processBuilder.redirectOutput(logFile);
        processBuilder.redirectError(logFile);

        try {
            // Start the application
            process = processBuilder.start();
            System.out.println("Spring Boot application started in the background.");
        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    public static void stopApp() {
        if (process != null && process.isAlive()) {
            process.destroy();
            System.out.println("Spring Boot application stopped.");
        } else {
            System.out.println("Application is not running.");
        }
    }
}


In [30]:
// START THE APP
RunSpringBootApp.startApp();

Spring Boot application started in the background.


In [33]:
// STOP THE APP
RunSpringBootApp.stopApp();

Spring Boot application stopped.


In [31]:
// CREATE LOGGER
import java.io.BufferedReader;
import java.io.File;
import java.io.FileReader;
import java.io.IOException;
import java.util.LinkedList;

public class LogReader {

    private static long lastReadLine = 0;

    public static void printNewLogs() {
        File logFile = new File("application.log");
        long currentLine = 0;

        try (BufferedReader reader = new BufferedReader(new FileReader(logFile))) {
            String line;
            while ((line = reader.readLine()) != null) {
                currentLine++;
                if (currentLine > lastReadLine) {
                    System.out.println(line);
                }
            }
            lastReadLine = currentLine;
        } catch (IOException e) {
            e.printStackTrace();
        }
    }

    public static void printLastNLogs(int n) {
        File logFile = new File("application.log");
        LinkedList<String> lastNLines = new LinkedList<>();

        try (BufferedReader reader = new BufferedReader(new FileReader(logFile))) {
            String line;
            while ((line = reader.readLine()) != null) {
                lastNLines.add(line);
                if (lastNLines.size() > n) {
                    lastNLines.removeFirst();
                }
            }
        } catch (IOException e) {
            e.printStackTrace();
        }

        for (String logLine : lastNLines) {
            System.out.println(logLine);
        }
    }
}



In [32]:
// VIEW LOG FILE (n last rows)
LogReader.printLastNLogs(60)

Standard Commons Logging discovery in action with spring-jcl: please remove commons-logging.jar from classpath in order to avoid potential conflicts

  .   ____          _            __ _ _
 /\\ / ___'_ __ _ _(_)_ __  __ _ \ \ \ \
( ( )\___ | '_ | '_| | '_ \/ _` | \ \ \ \
 \\/  ___)| |_)| | | | | || (_| |  ) ) ) )
  '  |____| .__|_| |_|_| |_\__, | / / / /
 =========|_|==============|___/=/_/_/_/
 :: Spring Boot ::                (v3.1.5)

2024-09-02T11:41:18.739+03:00  INFO 15930 --- [           main] c.e.a.AccountingDemoApplication          : Starting AccountingDemoApplication v3.0.x-SNAPSHOT using Java 17.0.10 with PID 15930 (/home/slava/projects/demo-jupyternb/res/application.jar started by slava in /home/slava/projects/demo-jupyternb/res)
2024-09-02T11:41:18.742+03:00  INFO 15930 --- [           main] c.e.a.AccountingDemoApplication          : No active profile set, falling back to 1 default profile: "default"
2024-09-02T11:41:19.920+03:00  INFO 15930 --- [           main] o.s.b.w.

In [6]:
// CREATE RMI (Remote Method Invocation) CLIENT
import java.lang.reflect.Method;
import java.rmi.RemoteException;
import java.rmi.registry.LocateRegistry;
import java.rmi.registry.Registry;
import java.lang.reflect.InvocationTargetException;

public class RmiEntityServiceClient {

    private static RmiEntityService getRmiEntityService() throws Exception {
        try {
            Registry registry = LocateRegistry.getRegistry("localhost", 1099);
            return (RmiEntityService) registry.lookup("RmiEntityService");
        } catch (RemoteException e) {
            throw new Exception("Failed to connect to RMI registry or lookup service", e);
        }
    }

    public static SerializableHttpResponse invokeMethod(String methodName, Object... args) throws Exception {
        try {
            RmiEntityService service = getRmiEntityService();
            Class<?>[] argTypes = new Class[args.length];
            for (int i = 0; i < args.length; i++) {
                argTypes[i] = args[i].getClass();
            }
            Method method = service.getClass().getMethod(methodName, argTypes);
            SerializableHttpResponse response = (SerializableHttpResponse) method.invoke(service, args);
            LogReader.printNewLogs();
            System.out.println("Response code: " + response.getStatusCode());
            System.out.println("Response body: " + response.getResponseBody());
            return response;
        } catch (NoSuchMethodException e) {
            throw new Exception("No such method found: " + methodName, e);
        } catch (IllegalAccessException e) {
            throw new Exception("Illegal access to method: " + methodName, e);
        } catch (InvocationTargetException e) {
            throw new Exception("Method invocation failed: " + methodName, e);
        } catch (RemoteException e) {
            throw new Exception("Remote method invocation failed", e);
        }
    }
}


In [ ]:
// SET LOCAL VARIABLES (OPTION 1)
String modelName = "entity_test";
String modelVersion = "1";
String entity = """
[{
    "fullName": "Peter Jackson",
    "department": "Accounting"
},
 {
    "fullName": "Nick Lane",
    "department": "Marketing"
}]   
""";

In [26]:
// SET LOCAL VARIABLES (OPTION 2)
String entity = """
{
  "employeeId": "7c24b516-57ba-11b2-884b-32c14073892a",
  "destination": "San Juan",
  "departureDate": "2024-09-01 11:55:16.479",
  "expenseList": [
    {
      "description": "other",
      "amount": "62.66"
    },
    {
      "description": "transportation",
      "amount": "27.65"
    }
  ],
  "advancePayment": "75.50",
  "amountPayable": "0.00"
}
  
""";
String modelName = "expense_report";
String modelVersion = "1";

In [10]:
// SAVE ENTITY MODEL
SerializableHttpResponse response = RmiEntityServiceClient.invokeMethod("saveEntityModel", entity, modelName, modelVersion);

2024-09-02T11:31:30.951+03:00  INFO 14950 --- [)-192.168.0.105] c.e.a.service.RmiEntityServiceImpl       : "POST https://cyoda-demo-app.kube.cyoda.org/api/treeNode/model/import/JSON/SAMPLE_DATA/expense_report/2 HTTP/1.1"
Response code: 200
Response body: "c5072f04-6905-11ef-884b-32c14073892a"


In [11]:
// LOCK ENTITY MODEL
RmiEntityServiceClient.invokeMethod("lockEntityModel", modelName, modelVersion);

2024-09-02T11:31:45.691+03:00  INFO 14950 --- [)-192.168.0.105] c.e.a.service.RmiEntityServiceImpl       : "PUT https://cyoda-demo-app.kube.cyoda.org/api/treeNode/model/expense_report/2/lock HTTP/1.1"
Response code: 200
Response body: 


In [27]:
// DELETE MODEL
RmiEntityServiceClient.invokeMethod("deleteEntityModel", modelName, modelVersion);

2024-09-02T11:39:18.997+03:00  INFO 14950 --- [)-192.168.0.105] c.e.a.service.RmiEntityServiceImpl       : "DELETE https://cyoda-demo-app.kube.cyoda.org/api/treeNode/model/expense_report/1 HTTP/1.1"
Response code: 404
Response body: {"errorMessage":"cannot find entityName=expense_report, version=1","timestamp":1725266359301}


In [24]:
// DELETE ALL ENTITIES BY MODEL
RmiEntityServiceClient.invokeMethod("deleteAllEntitiesByModel", modelName, modelVersion);

2024-09-02T11:38:49.483+03:00  INFO 14950 --- [)-192.168.0.105] c.e.a.service.RmiEntityServiceImpl       : "DELETE https://cyoda-demo-app.kube.cyoda.org/api/entity/TREE/expense_report/2 HTTP/1.1"
Response code: 200
Response body: [{"entityModelClassId":"c5072f04-6905-11ef-884b-32c14073892a","ids":["7df9636c-57bc-11b2-884b-32c14073892a"],"deleteResult":{"numberOfEntitites":1,"numberOfEntititesRemoved":1}}]


In [12]:
// SAVE ENTITY(S)
SerializableHttpResponse response = RmiEntityServiceClient.invokeMethod("saveEntity", entity, modelName, modelVersion);

2024-09-02T11:32:03.057+03:00  INFO 14950 --- [)-192.168.0.105] c.e.a.service.RmiEntityServiceImpl       : SAVE ENTITY REQUEST: "POST https://cyoda-demo-app.kube.cyoda.org/api/entity/new/JSON/TREE/expense_report/2 HTTP/1.1"
Response code: 200
Response body: [{"transactionId":"d46a8b80-6905-11ef-884b-32c14073892a","entityIds":["7df9636c-57bc-11b2-884b-32c14073892a"]}]


In [13]:
// GET ALL ENTITIES BY MODEL
SerializableHttpResponse response = RmiEntityServiceClient.invokeMethod("getAllEntitiesAsJson", modelName, modelVersion);

2024-09-02T11:32:11.124+03:00  INFO 14950 --- [)-192.168.0.105] c.e.a.service.RmiEntityServiceImpl       : "GET https://cyoda-demo-app.kube.cyoda.org/api/entity/TREE/expense_report/2?pageSize=100&pageNumber=1 HTTP/1.1"
Response code: 200
Response body: [{"id":"7df9636c-57bc-11b2-884b-32c14073892a","tree":{"employeeId":"7c24b516-57ba-11b2-884b-32c14073892a","destination":"San Juan","departureDate":"2024-09-01 11:55:16.479","expenseList":[{"description":"other","amount":"62.66"},{"description":"transportation","amount":"27.65"}],"advancePayment":"75.50","amountPayable":"0.00"}}]


In [14]:
// SET ENTITY ID
UUID id = UUID.fromString("7df9636c-57bc-11b2-884b-32c14073892a");

In [15]:
// GET BY ID
RmiEntityServiceClient.invokeMethod("getByIdAsJson", id);


2024-09-02T11:32:36.428+03:00  INFO 14950 --- [)-192.168.0.105] c.e.a.service.RmiEntityServiceImpl       : "GET https://cyoda-demo-app.kube.cyoda.org/api/entity/TREE/7df9636c-57bc-11b2-884b-32c14073892a HTTP/1.1"
Response code: 200
Response body: {"tree":{"employeeId":"7c24b516-57ba-11b2-884b-32c14073892a","destination":"San Juan","departureDate":"2024-09-01 11:55:16.479","expenseList":[{"description":"other","amount":"62.66"},{"description":"transportation","amount":"27.65"}],"advancePayment":"75.50","amountPayable":"0.00"},"meta":{"id":"7df9636c-57bc-11b2-884b-32c14073892a","type":"TREE","modelKey":{"legalEntityId":"CYODA","modelName":"expense_report","version":2}}}


In [16]:
// GET CURRENT STATE OF AN ENTITY
RmiEntityServiceClient.invokeMethod("getCurrentState", id);

2024-09-02T11:32:40.601+03:00  INFO 14950 --- [)-192.168.0.105] c.e.a.service.RmiEntityServiceImpl       : "GET https://cyoda-demo-app.kube.cyoda.org/api/platform-api/entity-info/fetch/lazy?entityClass=com.cyoda.tdb.model.treenode.TreeNodeEntity&entityId=7df9636c-57bc-11b2-884b-32c14073892a&columnPath=state HTTP/1.1"
Response code: 200
Response body: DRAFT


In [17]:
// GET AVAILABLE TRANSITIONS
RmiEntityServiceClient.invokeMethod("getListTransitions", id);

2024-09-02T11:32:44.065+03:00  INFO 14950 --- [)-192.168.0.105] c.e.a.service.RmiEntityServiceImpl       : "GET https://cyoda-demo-app.kube.cyoda.org/api/platform-api/entity/fetch/transitions?entityId=7df9636c-57bc-11b2-884b-32c14073892a&entityClass=com.cyoda.tdb.model.treenode.TreeNodeEntity HTTP/1.1"
Response code: 200
Response body: [DELETE, UPDATE, SUBMIT]


In [21]:
// LAUNCH TRANSITION
RmiEntityServiceClient.invokeMethod("launchTransition", id, "APPROVE_BY_MANAGER");

2024-09-02T11:34:28.948+03:00  INFO 14950 --- [ault-executor-4] c.e.a.p.CyodaCalculationMemberClient     : E-MAIL RECEIVED: ER with id: 7df9636c-57bc-11b2-884b-32c14073892a is pending manager's approval
2024-09-02T11:34:28.950+03:00  INFO 14950 --- [ault-executor-4] c.e.a.p.CyodaCalculationMemberClient     : << Sending event: success: true

2024-09-02T11:34:29.054+03:00  INFO 14950 --- [ault-executor-4] c.e.a.p.CyodaCalculationMemberClient     : << Received event: 
{
  "success" : true,
  "id" : "7e247b81-82bd-4538-9f88-e3ff132e8476"
}
2024-09-02T11:34:29.059+03:00  INFO 14950 --- [ault-executor-4] c.e.a.p.CyodaCalculationMemberClient     : << Received event: 
{
  "requestId" : "b6a59228-aecd-4cff-968a-4cb92cfc1a56",
  "entityId" : "7df9636c-57bc-11b2-884b-32c14073892a",
  "processorId" : "0a185a88-5bae-11ef-8cfd-229844f0e704",
  "processorName" : "calculateAmountPayable",
  "transactionId" : "2a44d8d0-6906-11ef-884b-32c14073892a",
  "payload" : {
    "type" : "TREE",
    "data" : {
  

In [ ]:
// DELETE ENTITY (model, version, id)
String idToDelete = id.toString();
RmiEntityServiceClient.invokeMethod("deleteEntityById", modelName, modelVersion, idToDelete);

In [23]:
// VIEW NEW LOGS
LogReader.printNewLogs();